# MNIST 手寫數字辨識 (MINST_Softmax_TF)

2017/07/02  
徐仕杰

## Outline

-  [Import Package & Functions](#import) 
-  [1. 先了解你的資料](#先了解你的資料) 
-  [2. 開始Machine Learning - Softmax regression](#開始Machine Learning)  
  -  [A. 定義參數](#定義參數) 
  -  [B. 設計一個Model從X預測Y](#設計一個) 
  -  [C. 選一個loss function,](#選一個loss) 
  -  [D. 選一個optimizer](#選一個o) 
  -  [E. 開始執行訓練](#開始執行) 
  -  [F. 算一下正確率](#算一下正)  
<br>    
-  [3. 好像沒用到validataion data 和 testing data?](#好像沒用到)
  

<a id='import'></a>
## Import Package & Functions

In [37]:
import pandas as pd
import os
from PIL import Image
import numpy as np
import lzma
import pickle
from IPython.display import display
import tensorflow as tf
from tfdot import tfdot
from tensorflow.contrib.tensorboard.plugins import projector

In [5]:
def showX(X):
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(-1,28,28).swapaxes(0,1).reshape(28,-1)
    display(Image.fromarray(int_X_reshape))

In [259]:
def showX2(X, rows=1):
    assert X.shape[0] % rows == 0
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(rows, -1,28,28).swapaxes(1,2).reshape(28*rows,-1)
    display(Image.fromarray(int_X_reshape))

<a id='先了解你的資料'></a>
## 1. 先了解你的資料 - MNIST
- 先把MNIST資料讀進來
- 看一下data size (總共50000+10000+10000筆資料)
- 看一下圖片長怎樣 
- 抓一張圖片出來 (正方形的～)
- 看一下一張圖片的資料 (1 rows × 784 columns) (長方形的？)
- 還原成 28 * 28 的格式 (28 rows x 784 rows)
- 把每個像素還原成0~255的格式 
- 把數字變成圖形

#### 先把MNIST資料讀進來

In [258]:
with lzma.open("mnist.pkl.xz", 'rb') as f:
    train_set, validation_set, test_set = pickle.load(f, encoding='latin1')

train_X, train_y = train_set
validation_X, validation_y = validation_set
test_X, test_y = test_set

#把Y label變成one-hot representation
train_Y = np.eye(10)[train_y]
test_Y = np.eye(10)[test_y]
validation_Y = np.eye(10)[validation_y]

#### 看一下data size

In [27]:
print('training data size:',len(train_X))
print('validataion data size:',len(validation_X))
print('testing data size:',len(test_X))

training data size: 50000
validataion data size: 10000
testing data size: 10000


#### 看一下圖片長怎樣

In [17]:
showX(train_X[:20])  #train_X[row,col], ex: train_X[0:20,0:784],train_X[0:20], train_X[:20]
print(train_y[:20])

[5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9]


#### 抓一張圖片出來

In [29]:
showX(train_X[:1])
print(train_y[:1])

[5]


#### 看一下這一張圖片的資料

In [31]:
pd.DataFrame(train_X[:1])

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 還原成正方形( 28 * 28 )  的大小

In [34]:
tmp = train_X[:1]
tmp = tmp.reshape(-1,28,28).reshape(28,-1)
pd.DataFrame(tmp)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.683594,0.101562,0.648438,0.996094,0.964844,0.496094,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.117188,0.140625,...,0.878906,0.671875,0.988281,0.945312,0.761719,0.250000,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.191406,0.929688,0.988281,...,0.363281,0.320312,0.320312,0.218750,0.152344,0.000000,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.070312,0.855469,0.988281,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.312500,0.609375,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0


#### 把每個像素還原成0~255的格式 

In [35]:
tmp2 = (tmp*255).clip(0,255).astype('uint8')
pd.DataFrame(tmp2)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,174,25,165,254,246,126,0,0,0,0
6,0,0,0,0,0,0,0,0,29,35,...,224,171,252,241,194,63,0,0,0,0
7,0,0,0,0,0,0,0,48,237,252,...,92,81,81,55,38,0,0,0,0,0
8,0,0,0,0,0,0,0,17,218,252,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,79,155,...,0,0,0,0,0,0,0,0,0,0


#### 為什麼只有一個數字, 而不是一般彩色圖片的RGB三個數字？
-> 因為MINST圖片是黑白的, 想像成R=G=B  
-> 不信的話, 找個調色盤試試看RGB三個數值都一樣時是不是黑白的~   
>每一筆資料都是一個28x28x1=784像素的byte資料，後面的x1是圖片只有黑白兩色也就是channel=1
如果圖源是RGB彩色的話就會是28x28x3，這第三維用來表示圖片有幾個"channel"

#### 把數字變成圖形

In [36]:
display(Image.fromarray(tmp2))

#### 要怎麼一次看所有圖片？
 - tensor: 把圖片的資料存成tf.Variable, 並寫入train_images.ckpt
 - metadata: 把圖片的label寫入metadata.tsv
 - embedding: 在Config裡用一個embedding物件關聯tensor & metadata

In [286]:
#50000張太多電腦跑不動, 先選10000張試試看
rnd_idx = np.random.choice(train_X.shape[0], 10000, replace=False) 
visualize_X = train_X[rnd_idx]
visualize_Y = train_y[rnd_idx]

#把每一張圖片資料存到tf.Variable
tfVar_X = tf.Variable(visualize_X, name='train_images')

#把圖片的label存到metadata.tsv
LOG_DIR = 'tb-log1'
metadata_file_path = os.path.join(LOG_DIR, 'metadata.tsv')
with open(metadata_file_path, 'w') as metadata_file:
    for row in range(len(visualize_Y)):
        c=visualize_Y[row]
        metadata_file.write('{}\n'.format(c))
        

In [ ]:
with tf.Session() as sess:
    #把圖片的variable存在 train_images.ckpt
    saver = tf.train.Saver([tfVar_X])
    sess.run(tfVar_X.initializer)
    saver.save(sess, os.path.join(LOG_DIR, 'train_images.ckpt'))

    #在config裡面用一個embedding關聯 tensor & its metadata
    config = projector.ProjectorConfig()
    # One can add multiple embeddings.
    embedding = config.embeddings.add()
    embedding.tensor_name = tfVar_X.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = metadata_file_path
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)

# for calling the tensorboard you should be in that drive and call the entire path
#tensorboard --logdir=tb-log1 --port=6006

In [ ]:
!tensorboard --logdir=tb-log1 --port=6006

Starting TensorBoard b'41' on port 6006
(You can navigate to http://192.168.43.126:6006)


![mnisttensorboard](tb-log1/mnist.png)

#### 更詳細的介紹文件
http://colah.github.io/posts/2014-10-Visualizing-MNIST/  
https://github.com/anujshah1003/Tensorboard-examples/blob/master/mnist-tensorboard-embeddings-1.py  
https://www.youtube.com/watch?v=e2w-kOVHNQ4  
https://www.tensorflow.org/versions/r0.12/how_tos/embedding_viz/#tensorboard-embedding-visualization  

<a id='開始Machine Learning'></a>
## 2. 開始Machine Learning - Softmax regression 
- 因為我們的目的是讓電腦從一堆圖片的數字中, 正確判斷這張圖片是哪個數字,  
所以運算的結果最好可以直接告訴我們判斷的結果, ex: 哪一個數字的機率最大？  
<br>
- 基本上就是用* $ e ^ {W x +b} $ 的比例 *來計算機率,  
其中 x 是長度 784 的向量（圖片）， W 是 10x784矩陣，加上一個長度為 10 的向量b。 算出來的十個數值，依照比例當成預估的機率。  
<br>
- Machine Learning ABC <- 這個很重要, 老闆會問你
    -  [A. 定義參數](#定義參數) 
    -  [B. 設計一個Model從X預測Y](#設計一個) 
    -  [C. 選一個loss function,](#選一個loss) 
    -  [D. 選一個optimizer](#選一個o) 
    -  [E. 開始執行訓練](#開始執行) 
    -  [F. 算一下正確率](#算一下正)  

<a id='定義參數'></a>
### A. 定義參數(Placeholder, Variable, Constant)
tips: 把要餵進Model的資料X,Y定義成placeholder, 把要讓電腦幫忙找的權重W,B定義成Variable

In [97]:
tf.reset_default_graph()
# 輸入的 placeholder
X = tf.placeholder(tf.float32, shape=[None, 784], name="X")
# 權重參數，為了計算方便和一些慣例（行向量及列向量的差異），矩陣乘法的方向和上面解說相反
W = tf.Variable(tf.zeros([784, 10]), name='W')
b = tf.Variable(tf.zeros([10]), name='b') # 這裡可以看成是列向量

#tfdot() #我們都是有tensorflow sense的人, 不要再畫圖了好嗎~~~

<a id='設計一個'></a>
###  B. 設計一個Model從X預測Y  ( $y=softmax(WX+B)$  )
- 簡單迴歸 $Y=WX+B$
- softmax迴歸(logistic迴歸) $ y=softmax(WX+B)$

In [71]:
?tf.nn.softmax

In [270]:
Y= tf.add(x= tf.matmul(X, W, name='e'), y=b, name='Y')
Y_softmax = tf.nn.softmax(Y, name='Y_softmax') #這個是one-hot的格式
Y_softmax_argmax = tf.argmax(Y_softmax, axis=1) #從one-hot的格式找出最大那個
#Y_softmax = tf.div(Y, tf.reduce_sum(Y, axis=1, keep_dims=True), name="Y_softmax")

correct_prediction = tf.equal(Y_softmax_argmax, tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
#tfdot()

<a id='選一個loss'></a>
###  C. 選一個loss function, 當作Machine learning的目標
tips: 最常用的就是MSE, 讓電腦找到的Y_Predict 和 真實資料 Y_True的差距最小. 但專業的都會用交叉熵cross_entorpy $-log(\Pr(Y_{true}))$

In [99]:
# 真正的 Y
Y_ = tf.placeholder(tf.float32, shape=[None, 10], name="Y_")
#和算出來的 Y 來做 cross entropy
cost_crossentropy = tf.reduce_mean(-tf.reduce_sum(Y_*tf.log(Y_softmax), axis=1))
# or
#cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_, logits=Y))
#tfdot()

<a id='選一個o'></a>
### D. 選一個optimizer, 根據Data和我們訂的目標找參數W, B
- 就先用我們上次講的梯度下降Optimizer吧, 總之可以幫我們找到最小的cost function就好了~

In [100]:
optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(cost_crossentropy)

<a id='開始執行'></a>
### E. 開始執行訓練

In [102]:
#先創造一個session, 然後記得要init variable
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()


In [103]:
#訓練1000次, 每次用50張圖片
for i in range(1000):
    rnd_idx = np.random.choice(train_X.shape[0], 50, replace=False) 
    optimizer.run(feed_dict={X: train_X[rnd_idx], Y_:train_Y[rnd_idx]})

In [217]:
# 抓出前五筆訓練資料的預測結果 Y_softmax (注意：這個是one-hot的格式)
predict_y_raw = Y_softmax.eval(feed_dict={X: train_X[:5]})
predict_y_df = pd.DataFrame(predict_y_raw)
predict_y_df

,0,1,2,3,4,5,6,7,8,9
0,0.006956,7.461185e-05,0.006922,0.297928,1.491603e-05,0.669682,0.001714,0.008905,0.006047,0.001756
1,0.998987,1.031839e-08,0.000032,0.000049,6.891966e-08,0.000851,0.000012,0.000015,0.000047,0.000006
2,0.001284,2.677838e-04,0.016490,0.070300,8.539041e-01,0.001533,0.006767,0.004461,0.004630,0.040364
3,0.000035,9.625692e-01,0.015424,0.003343,1.948921e-04,0.000342,0.000297,0.000694,0.016944,0.000157
4,0.000020,3.193711e-04,0.000043,0.000125,8.192862e-02,0.000498,0.000134,0.020453,0.007837,0.888643


In [219]:
# transfer one hot prediciton to normal format by argmax (為了方便和正確答案比較, 所以從one-hot裡面找最大的那個答案)
prediction = predict_y_df.apply(np.argmax,axis=1)
print('predict: ',prediction.values)
showX(train_X[:5])
print('answer: ',train_y[:5])

predict:  [5 0 4 1 9]


answer:  [5 0 4 1 9]


In [220]:
#argmax in tensorflow (也可以直接讓tensorflow從one-hot裡面找出最大那個答案, 回頭看一下Y_softmax_argmax這個定義~)
predicton = Y_softmax_argmax.eval(feed_dict={X: train_X[21:30]})
print('prediction:',predicton)
showX(train_X[21:30])
print('answer:',train_y[21:30])

prediction: [0 9 1 3 2 4 3 7 7]


answer: [0 9 1 1 2 4 3 2 7]


<a id='算一下正'></a>
### F. 算一下正確率(Accuracy)

In [247]:
#回頭看一下correct_prediction的定義
_ = correct_prediction.eval({X: train_X[21:30] , Y_: train_Y[21:30]})
print('第21~30筆資料的比對結果:',pd.DataFrame(_))

第21~30筆資料的比對結果:        0
0   True
1   True
2   True
3  False
4   True
5   True
6   True
7  False
8   True


In [248]:
_=accuracy.eval(feed_dict={X: train_X[21:30] , Y_: train_Y[21:30]})
print('第21~30筆筆資料的accuracy:',_)

第21~30筆筆資料的accuracy: 0.777778


In [249]:
# predict all X and get the accuracy
_ = accuracy.eval(feed_dict={X: train_X , Y_: train_Y})
print('Accuracy:',_)

Accuracy: 0.9143


<a id='好像沒用到'></a>
## 3. 好像沒用到validataion data 和 testing data?
- Training Data: 訓練Model
- Validataion Data: 訓練Model的時候, 同步監控目前模型的好壞
- Testing Data: 訓練結束後, 評估模型的好壞

#### E-2. 開始執行訓練(Training Data + Validataion Data)

In [242]:
for t in range(10):
    for i in range(100):
        rnd_idx = np.random.choice(train_X.shape[0], 50, replace=False)
        optimizer.run(feed_dict={X: train_X[rnd_idx], Y_:train_Y[rnd_idx]})
    a = accuracy.eval({X: validation_X , Y_: validation_Y})
    print (t,':', a)

0 : 0.9182
1 : 0.9182
2 : 0.9185
3 : 0.9202
4 : 0.9191
5 : 0.9196
6 : 0.9202
7 : 0.9191
8 : 0.9197
9 : 0.9195


#### F-2. 算一下正確率Accuracy (用Testing Data)
- unseen data
- generalize from the training set to the test set
- 泛化

In [251]:
_ = accuracy.eval({X: test_X , Y_: test_Y})
print('Accuracy:',_)

Accuracy: 0.9173


In [ ]:
sess.close()